In [126]:
import numpy as np
from Coursework.ANNModel import model as mod
from Coursework.ANNModel import data as d
from Coursework.PSO import pswarm as ps
from Coursework.PSO import psobehaviour as psh
from Coursework.PSO import history as hs
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

data = d.Data("./Data/1in_tanh.txt")
decimals = 'linear'
X = data.get_rows()
y = data.get_output()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=14)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
#train the model
model = mod.ANN()
model.set_training_input(X_train, y_train)
model.add(mod.Layer(4, activation="sigmoid"))
model.add(mod.Layer(1, activation="null"))
model.compile()

model_history = hs.PSOHistory(model)

Model Compiled!


In [129]:
# Cubic (10 runs) mean fitness: 252
Cubic_pso_config_mean = [65.07982407, 4.95690714, 0.28949701, 0.54421663, 0.39890445, 1.42315492, 1.18854163]

# Tanh (10 runs) mean fitness: 726 
Tanh_pso_config_mean = [57.81506585, 4.49521216, 0.14795705, 1.12579856, 0.3560856 , 1.03022577, 1.22604699]

In [130]:
pso = ps.PSO(swarm_size=100, num_informants=6, boundary_policy=psh.BoundaryPolicy.RANDOMREINIT, termination_policy=[psh.TerminationPolicy.ITERATIONS], termination_args={'max_iter': 400}, alpha=0.61, beta=1.3, gamma=1.4, delta=1.3, epsilon=0.6)
pso = pso.decode_vec(Cubic_pso_config_mean)

print(pso.swarm_size)

65


In [131]:
pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.evaluate_fitness)

train ={
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

test = {
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

for i in range(10):
    fitness = pso.run()
    # Evaluating on training set
    model.set_training_input(X_train, y_train)
    train['fitnessList'].append(model.evaluate_fitness(pso.best.location))
    score = data.score(model, decimals, atol=0.07)
    train['scoreA'].append(score[0])
    train['scoreB'].append(score[1])
    train['lossList'].append(model.loss)
    # Evaluating on testing set
    model.set_training_input(X_test, y_test)
    test['fitnessList'].append(model.evaluate_fitness(pso.best.location))
    score = data.score(model, decimals, atol=0.07)
    test['scoreA'].append(score[0])
    test['scoreB'].append(score[1])
    test['lossList'].append(model.loss)

print("Testing on train - A: ", np.mean(train['scoreA']), " B: ", np.mean(train['scoreB']), " Fitness: ", round(np.mean(train['fitnessList']), 3), " Loss: ", round(np.mean(train['lossList']), 3))
print("Testing on test - A: ", np.mean(test['scoreA']), " B: ", np.mean(test['scoreB']), " Fitness: ", round(np.mean(test['fitnessList']), 3), " Loss: ", round(np.mean(test['lossList']), 3))

Testing on train - A:  0.8611940298507463  B:  0.8611940298507463  Fitness:  2.2643162608206555e+31  Loss:  0.004
Testing on test - A:  0.8757575757575757  B:  0.8757575757575757  Fitness:  5.9650407774290445e+31  Loss:  0.004


In [132]:
pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.evaluate_fitness)
fitness = pso.run()
print(fitness.location)
print(fitness.fitness)

[ 0.3978106  -0.39990203 -0.49049597  0.3954873  -0.02101697 -0.23048742
  0.80491243 -0.45975884 -0.96148272  0.48277859  0.04597148 -0.03371191
  0.54988909  0.4891192  -0.80464713]
4536114.258151777


In [133]:
pso_best = pso.best.location
model.decode_vec(pso_best)

In [134]:
#model.layers[-1].bias = np.array([0.0])
#model.layers[-1].weights = np.array([[1.0]])


In [135]:
#run model
model.set_training_input(X_test, y_test)
model.one_pass()


In [136]:
np.around(model.y, 4)

array([-0.52, -0.22, -0.3 , -0.12,  0.1 ,  0.4 ,  0.64, -0.2 ,  0.82,
        0.3 , -0.96,  0.8 , -0.64,  0.46,  0.94,  0.38,  0.04, -0.84,
       -0.42, -0.88, -0.32,  0.28, -0.24, -0.6 ,  0.  ,  0.98, -0.86,
        0.48,  0.08, -0.16, -0.54,  0.58,  0.16])

In [53]:
print('activation: ', model.layers[-1].activation)
print('bias: ', model.layers[-1].bias)
print('weights: ', model.layers[-1].weights)

activation:  ActivationFunction.NULL
bias:  [0.14314255]
weights:  [[ 0.39141947]
 [ 0.54853014]
 [-0.23735302]
 [-0.92583363]]


In [54]:
all_fitness = model_history.historical_particle_fitness()
all_locations = model_history.historical_particle_location()

In [55]:
len(model_history.vec_history[::10])

2600

In [56]:
len(model_history.particle_location[0][::10])

260

In [115]:
# Cubic Most fit model: 6847
Cubic_ann_params = np.array([ 4.06948631e+00, -7.29125756e-01,  5.73721712e-01, -4.97778569e-01, 1.21867021e-01, -4.07830431e-01,  9.23118308e-01,  9.96763804e-01, -3.53176479e-01,  2.82162360e+00, -6.77411971e-01, -6.33183382e-01, 3.17935306e-01, -1.91733120e-01,  2.57710600e-03])

#Linear Most fit model: 552335863302
Linear_ann_params = np.array([0.37873092, -0.77783744, -0.15640582,  0.05812352,  0.38093837, -0.51612184,  0.91428793,  0.19934679, 0.67594604,  0.06668804, -0.35705256, -0.36102463,  0.58055201, -0.14048983,  0.45992348])

#Sine Most fit model: 329
Sine_ann_params = np.array([ 3.50680037,  0.2952162 ,  0.71664711, -0.95665899, -0.9055195, -0.27101866,  0.41973435,  0.36889121, -0.28896977,  3.3595307, 0.42623047,  0.30804742,  0.62065839, -0.73504326,  0.70304275])

#Tanh Most fit model: 20498
Tanh_ann_params = np.array([ 4.06311365, -0.9872686 ,  0.90773279, -0.96842173,  0.43744964,
        0.88650714,  0.99799899, -0.99414366, -0.83390916,  1.87425395, -0.55012036, -0.74588025,  0.28643476,  0.90210219, -0.37562391])

In [116]:
model.decode_vec(Cubic_ann_params)
model.one_pass()

# Evaluating on training set
model.set_training_input(X_train, y_train)
train['fitnessList'].append(model.evaluate_fitness(Linear_ann_params))
score = data.score(model, decimals, atol=0.07)
train['scoreA'].append(score[0])
train['scoreB'].append(score[1])
train['lossList'].append(model.loss)
# Evaluating on testing set
model.set_training_input(X_test, y_test)
test['fitnessList'].append(model.evaluate_fitness(Cubic_ann_params))
score = data.score(model, decimals, atol=0.07)
test['scoreA'].append(score[0])
test['scoreB'].append(score[1])
test['lossList'].append(model.loss)

print("Testing on train - A: ", np.mean(train['scoreA']), " B: ", np.mean(train['scoreB']), " Fitness: ", round(np.mean(train['fitnessList']), 3), " Loss: ", round(np.mean(train['lossList']), 3))
print("Testing on test - A: ", np.mean(test['scoreA']), " B: ", np.mean(test['scoreB']), " Fitness: ", round(np.mean(test['fitnessList']), 3), " Loss: ", round(np.mean(test['lossList']), 3))


Testing on train - A:  0.47082767978290363  B:  0.47082767978290363  Fitness:  496.456  Loss:  0.019
Testing on test - A:  0.46524064171123  B:  0.46524064171123  Fitness:  745.398  Loss:  0.015


In [80]:
1/model.loss

6847.0587291622405

In [82]:
model.evaluate_fitness(Cubic_ann_params)

6847.058829162241